# $$ Interacting With the Chatbot $$

1. Import necessary libraries:
   - `tkinter` for creating the GUI.
   - Your pre-trained chatbot model.
   - Other relevant libraries for processing user input and generating responses.

2. Create the GUI structure:
   - Design the user interface with an input box for user messages and an area for displaying chat history.

3. Define functions for processing user input:
   - When the user enters a message, preprocess it (e.g., tokenize and lemmatize) to prepare it for the model.
   - Use your pre-trained model to predict the intent of the user's message.
   - Randomly select a response from the list of responses for that intent.

4. Display the chat:
   - Show the user's input and the chatbot's response in the chat history area.
   - Update the chat history as the conversation progresses.

5. Handle user interactions:
   - Allow the user to enter messages and receive responses.
   - Implement a way to end the conversation or exit the GUI.

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

### Code Documentation

**Step 1: Import Necessary Libraries**

- We begin by importing essential libraries for our chatbot.
- `nltk` provides natural language processing tools.
- `WordNetLemmatizer` from `nltk.stem` is used for word lemmatization.
- `pickle` is employed for data serialization.
- `numpy` is imported as `np` for numerical operations.

These libraries are crucial for text processing and data manipulation in our chatbot.

**Step 2: Initialize Lemmatizer**

- We initialize a WordNet lemmatizer (`lemmatizer`) to convert words to their base or canonical forms.
- Lemmatization helps standardize words for effective text analysis.

**Step 3: Additional Libraries**

- We import `numpy` as `np` to work with numerical data, which is often required in machine learning applications.


In [2]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random

### Code Documentation

**Step 1: Load Pre-Trained Model by OurSelf**

- We load a pre-trained chatbot model using the `load_model` function from the Keras library. This model has been saved previously and is stored in the 'chatbot_model.h5' file.
- The loaded model is ready to make predictions and generate responses based on user input.

**Step 2: Import Necessary Libraries**

- We import additional libraries required for our chatbot.
- `json` is used for handling structured data.
- `random` is employed for generating random responses from a list of possible replies.



In [3]:
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

### Code Documentation

**Step 1: Load Intents and Vocabulary Data**

- We load the intents and vocabulary data used in our chatbot.
- `intents` is loaded from a JSON file named 'intents.json,' which contains predefined intents, patterns, and responses for the chatbot.
- `words` and `classes` are loaded using the `pickle` library from 'words.pkl' and 'classes.pkl' files, respectively. These files store preprocessed data used for training the chatbot.

In [4]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for words in sentence_words]
    return sentence_words

### Code Documentation

**Clean Up Sentence Function**

- This code defines a function called `clean_up_sentence`.
- The function takes a sentence as input and prepares it for further processing.
- It tokenizes the input sentence, breaking it down into individual words.
- Then, it lemmatizes each word, reducing them to their base or canonical forms.
- The processed words are converted to lowercase for consistency.
- The function returns a list of cleaned and standardized words from the input sentence.

This function is essential for text preprocessing, ensuring that user input is in a consistent format for analysis and response generation in our chatbot.


In [5]:
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)

    return(np.array(bag))

### Code Documentation

**Bag of Words Function**

- This code defines a function named `bag_of_words`.
- The function takes a sentence, a list of words (vocabulary), and an optional `show_details` flag as inputs.

**Step 1: Tokenization**
- The input sentence is tokenized using the `clean_up_sentence` function, which processes and standardizes the words in the sentence.

**Step 2: Vocabulary Matrix**
- A bag of words (BoW) matrix is created, represented as a list of zeros with the same length as the vocabulary words.
- Each element in the `bag` list corresponds to a word in the vocabulary.

**Step 3: Filling the Bag**
- For each word in the tokenized sentence, the code checks if it exists in the vocabulary.
- If a word is found in the vocabulary, the corresponding position in the `bag` list is set to 1, indicating its presence.
- If the `show_details` flag is True, it prints a message for each word found in the bag.

**Step 4: Return the BoW Matrix**
- The function returns the BoW matrix as a NumPy array, where each element represents the presence (1) or absence (0) of a word from the sentence in the vocabulary.

In [6]:
def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

### Code Documentation

**Predict Class Function**

- This code defines a function called `predict_class`.
- The function is used to predict the intent class of a user's input sentence based on the chatbot's model.

**Step 1: Encoding the Sentence**
- The input sentence is encoded into a bag of words (BoW) matrix using the `bag_of_words` function. This step converts the sentence into a numerical representation for processing.

**Step 2: Model Prediction**
- The BoW matrix is passed to the chatbot model, and it predicts the probabilities of each intent class for the given input.
- Probabilities are calculated for all intent classes.

**Step 3: Filtering Predictions**
- Predictions with probabilities below a defined threshold (ERROR_THRESHOLD) are filtered out. This threshold determines the minimum confidence level for considering an intent.
- The filtered results are sorted by probability in descending order.

**Step 4: Return Intent and Probability**
- The function returns a list of intent-class and probability pairs.
- Each pair includes the predicted intent and its corresponding probability.
- The list is sorted in descending order of probability, starting with the most likely intent.

This function is crucial for determining the user's intent based on their input sentence, allowing the chatbot to provide relevant responses.


In [7]:

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
            
    return result

### Code Documentation

**Get Response Function**

- This code defines a function called `getResponse`.
- The function is responsible for selecting and returning a response based on the predicted intent.

**Step 1: Identify the Intent**
- The function takes a list of predicted intents (`ints`) and the intents JSON data (`intents_json`) as inputs.
- It identifies the intent with the highest probability from the predictions.

**Step 2: Retrieve Responses**
- The code retrieves the list of intents and responses from the provided JSON data.
- It iterates through the intents to find a match between the predicted intent and the intents in the JSON data.

**Step 3: Random Response Selection**
- Once a match is found, the function randomly selects a response from the list of responses associated with the matched intent.
- This selected response is returned as the chatbot's reply.

This function is vital for generating meaningful and contextually relevant responses to user inputs based on the predicted intent.


# GUI for ChatBot using Tkinter

In [8]:
import tkinter
from tkinter import *

def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatBox.config(state=NORMAL)
        ChatBox.insert(END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="#446665", font=("Verdana", 12 )) 

        ints = predict_class(msg)
        res = getResponse(ints, intents)
        
        ChatBox.insert(END, "Bot: " + res + '\n\n')           
        ChatBox.config(state=DISABLED)
        ChatBox.yview(END)

### Code Documentation

**Message Sending Function**

- This code defines a function called `send`, which is executed when the user sends a message in the chat interface.
- The function performs several actions to manage and display the conversation.

**Step 1: Get User Message**
- The function retrieves the message entered by the user from the input box.
- It trims leading and trailing whitespace to ensure that the message is clean and does not contain unnecessary spaces.

**Step 2: Clear Input Box**
- The input box is cleared to prepare it for the next user message.

**Step 3: Display User Message**
- If the user's message is not empty, it is displayed in the chat box with "You:" as the sender. The message is formatted for presentation.

**Step 4: Predict Intent**
- The user's message is passed to the `predict_class` function to determine its intent based on the chatbot's model.

**Step 5: Generate and Display Bot Response**
- The function uses the predicted intent to select and obtain a response from the chatbot's predefined intents.
- The bot's response is displayed in the chat box with "Bot:" as the sender.

**Step 6: Configure Chat Box**
- The chat box is configured to control its appearance and scrolling behavior, ensuring a user-friendly chat interface.

This function manages the flow of the conversation in the graphical user interface, facilitating user input and chatbot responses.


In [9]:
root = Tk()
root.title("Chatbot")
root.geometry("400x500")
root.resizable(width=FALSE, height=FALSE)

''

### Code Documentation

**Initialize Chatbot GUI**

- This code creates and initializes the graphical user interface (GUI) for the chatbot.
- It uses the `Tk()` function to create a new GUI window.

**Step 1: Set Window Title**
- The GUI window is given a title, which is displayed in the window's title bar. In this case, it's set to "Chatbot."

**Step 2: Define Window Dimensions**
- The `geometry` method is used to set the dimensions of the GUI window. In this code, it's set to a width of 400 pixels and a height of 500 pixels.

**Step 3: Make Window Non-Resizable**
- The `resizable` method is used to configure the window's resizing behavior. In this code, the window is set to be non-resizable, meaning users cannot change its dimensions.

This code initializes the GUI, defines its appearance, and ensures it meets the desired dimensions and user interaction behavior.


In [10]:
#Create Chat window
ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatBox.config(state=DISABLED)

### Code Documentation

**Create Chat Window**

- This code segment creates a chat window within the GUI for the chatbot.

**Step 1: Initialize Text Widget**
- The `Text` widget is used to create a multi-line text box in the GUI, which will serve as the chat window.

**Step 2: Configure Text Box**
- `bd=0` removes the border around the text box.
- `bg="white"` sets the background color of the text box to white.
- `height="8"` specifies the initial height of the text box, allowing multiple lines of text.
- `width="50"` sets the initial width of the text box, indicating the number of characters it can display in a line.
- `font="Arial"` sets the font style of the text.

**Step 3: Disable Initial Editing**
- `ChatBox.config(state=DISABLED)` disables the ability to edit the text box initially. This ensures that the user cannot interact with it until it's enabled.

This code segment establishes the chat window's appearance and behavior, readying it to display conversations between the user and the chatbot.


In [11]:
#Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set


### Code Documentation

**Bind Scrollbar to Chat Window**

- This code binds a scrollbar to the chat window, enabling users to scroll through the conversation history.

**Step 1: Create Scrollbar**
- A scrollbar widget is created using the `Scrollbar` function. This scrollbar will allow vertical scrolling.

**Step 2: Set Scroll Command**
- The `command` parameter of the scrollbar is set to `ChatBox.yview`, which means the scrollbar's behavior will be controlled by the chat text box (`ChatBox`).
- The `cursor` parameter is set to "heart" to specify the cursor appearance over the scrollbar.

**Step 3: Configure Chat Text Box**
- The `yscrollcommand` attribute of the chat text box (`ChatBox`) is set to `scrollbar.set`. This binds the scrollbar to the chat box for vertical scrolling synchronization.

By binding the scrollbar to the chat window, users can easily navigate through the conversation history, especially when it exceeds the visible area of the text box.


In [12]:
#Create Button to send message
SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#f9a602", activebackground="#3c9d9b",fg='#000000',
                    command= send )

### Code Documentation

**Create Send Button**

- This code segment creates a "Send" button within the chatbot's graphical user interface (GUI), allowing users to send their messages.

**Step 1: Initialize Button**
- The `Button` widget is used to create a button in the GUI.

**Step 2: Configure Button Appearance**
- `font=("Verdana", 12, 'bold')` sets the font style and size for the button's text.
- `text="Send"` specifies the text displayed on the button.
- `width="12"` sets the button's width.
- `height=5` determines the button's height.
- `bd=0` removes the button's border.
- `bg="#f9a602"` sets the background color of the button.
- `activebackground="#3c9d9b"` specifies the background color when the button is clicked.
- `fg='#000000'` sets the foreground color (text color) of the button.
- `command=send` associates the button with the `send` function, defining the action to be performed when the button is clicked.

This code creates a button that, when clicked, triggers the `send` function to send the user's message in the chat interface.


In [13]:
#Create the box to enter message
EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)

### Code Documentation

**Create Message Input Box**

- This code creates an input box within the chatbot's graphical user interface (GUI) for users to enter their messages.

**Step 1: Initialize Text Widget**
- The `Text` widget is used to create a text input box in the GUI.

**Step 2: Configure Input Box Appearance**
- `bd=0` removes the border around the input box.
- `bg="white"` sets the background color of the input box to white.
- `width="29"` specifies the initial width of the input box, allowing for a certain number of characters in a line.
- `height="5"` determines the initial height of the input box, enabling multiple lines of text to be entered.
- `font="Arial"` sets the font style of the text in the input box.

**Step 3: Bind Action (Optional)**
- The code includes a line that is currently commented out (`#EntryBox.bind("<Return>", send)`). This line is used to bind an action (the `send` function) to the Enter key press. It's currently commented, but it can be used to trigger the message sending action when the user presses Enter.

This code segment creates an input box where users can type their messages for the chatbot, facilitating user interaction.


In [14]:
root.mainloop()

### Code Documentation

**Run the GUI Main Loop**

- This code initiates the main loop for the graphical user interface (GUI) of the chatbot.

**Step 1: `root.mainloop()`**
- The `mainloop` function is invoked on the `root` object, which represents the main GUI window.
- This function is crucial for running the GUI application and continuously processing user interactions and events.

By calling `root.mainloop()`, the chatbot's GUI becomes responsive and ready to interact with users, continuously updating the interface and handling user actions.
